# Custom Neural Network Training
In this tutorial we will be training a custom neural networks using the NiftyTorch library.  

We will build a module which is a combination of both **ShuffleNet** and **ResNet** architecture by combining the shuffleunit from ShuffleNet and bottleneck unit from ResNet.  

We will start by importing the neccessary modules.

In [1]:
from niftytorch.Layers.Convolutional_Layers import ShuffleUnit
from niftytorch.Layers.Convolutional_Layers import Bottleneck
from niftytorch.Layers.Convolutional_Layers import conv3x3
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.optim as optim
from niftytorch.Loader.Classification_DataLoader import ImageFolder
import torch
from torchvision import transforms
from niftytorch.Models.Trainer import train_model
import os

<p>The type of architecture we are going to follow is the following:
SH1 (ADD) --> SH2 (CONCAT) --> SH3(ADD) --> CN1 --> BN1 --> CN2
    
Here SH refers to ShuffleNet Unit with Addition or Concation of grouped convolutions tensors.
     BN refers to Bottleneck layer from resent.
     CN refers to a 3x3 Convolutional Layer.
</p>

<p>We start by defining the network architecture in pytorch</p>

In [ ]:
class Neural_Network(nn.Module):
    def __init__(self):
        super(Neural_Network, self).__init__()
        # This is an example of how to make a hypothetical network using parts of ShuffleNet and ResNet
        self.sh1 = ShuffleUnit(in_channels = 1, out_channels = 12, groups=1,grouped_conv=True, combine='add',compresstion_ratio = 4)
        self.sh2 = ShuffleUnit(n_channels = 12, out_channels = 36, groups=2,grouped_conv=True, combine='concat',compresstion_ratio = 2)
        self.sh3 = ShuffleUnit(in_channels = 36, out_channels = 36, groups=2,grouped_conv=True, combine='add',compresstion_ratio = 2)
        self.cn1 = conv3x3(in_planes = 36,out_planes = 4,stride = 2) 
        self.bn1 = Bottleneck(inplanes = 4, planes = 4, stride = 1, downsample=None, groups=1,base_width=64, dilation=1, norm_layer=None,expansion = 1)
        self.cn2 = conv3x3(in_planes = 4,out_planes = 2,stride = 2) 
        self.classifier = nn.Sequential(
        nn.Linear(1024,128),
        nn.ReLU(inplace=True),
        nn.Linear(128,32),
        nn.ReLU(inplace=True),
        nn.Linear(32, num_classes),
        )
    def forward(self, x):
        x = self.sh1(x)
        x = self.sh2(x)
        x = self.sh3(x)
        x = self.cn1(x)
        x = self.bn1(x)
        x = self.cn2(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

gamma = 0.1
batch_size = 64
learning_rate = 1e-3
step_size = 15
num_classes = 2
loss = nn.CrossEntropyLoss()
num_workers = 4
data_transforms = data_transforms
filename_label = 'Subject'
class_label = 'disease'
image_scale = 64
device = torch.device('cuda:7')
image_datasets = {x: ImageFolder(os.path.join(data_folder,x),data_csv,data_transforms,filename_label = filename_label,class_label = class_label,common = image_scale) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,shuffle=True, num_workers=num_workers) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

model = Neural_Network()
model = model.to(device)
scheduler = lr_scheduler.StepLR
optimizer = optim.Adam
optimizer_ft = optimizer(model.parameters(), lr=learning_rate)
exp_lr_scheduler = scheduler(optimizer_ft, step_size=step_size, gamma=gamma)
train_model(model,loss,optimizer_ft,exp_lr_scheduler,dataloaders,dataset_sizes,device) 

<p>Define the location where the data is stored, the format required to read is given in getting start ipython notebook</p>

In [ ]:
data_transforms = transforms.Compose([transforms.ToTensor()]) 
data_folder = "/example/farshid/img/data/StudyName"
data_csv = "/example/farshid/img/data/StudyName/labels.csv"

Define the set of external parameters needed to run the train_model function

In [ ]:
gamma = 0.1
batch_size = 64
learning_rate = 1e-3
step_size = 15
num_classes = 2
loss = nn.CrossEntropyLoss()
num_workers = 4
data_transforms = data_transforms
filename_label = 'Subject'
class_label = 'disease'
image_scale = 64
device = torch.device('cuda:7')

Define the dataloader for the training the custom network

In [ ]:
image_datasets = {x: ImageFolder(os.path.join(data_folder,x),data_csv,data_transforms,filename_label = filename_label,class_label = class_label,common = image_scale) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,shuffle=True, num_workers=num_workers) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

<p>Initialize the network structure and weights</p>

In [ ]:
model = Neural_Network()
model = model.to(device)

Define the scheduler and the optimizer

In [ ]:
scheduler = lr_scheduler.StepLR
optimizer = optim.Adam
optimizer_ft = optimizer(model.parameters(), lr=learning_rate)
exp_lr_scheduler = scheduler(optimizer_ft, step_size=step_size, gamma=gamma)

Finally 